In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [4]:
items_urls = []

In [8]:
for i in tqdm(range(1, 272)):
    if i == 1:
        end = '/'
    else:
        end = f'&page={i}'
    link = f'https://kino.mail.ru/series/all/?order=rate_count&year=1999&year=2024{end}'
    content = requests.get(link).content
    soup = BeautifulSoup(content, 'lxml')
    series = soup.find_all(class_='link link_inline link-holder link-holder_itemevent link-holder_itemevent_small')

    for item in series:
        item_page_url = item.get('href')
        items_urls.append('https://kino.mail.ru' + item_page_url)

100%|██████████| 271/271 [03:44<00:00,  1.21it/s]


In [9]:
urls = pd.DataFrame({'url':items_urls})
urls.to_csv('urls_mail_ru.csv')

items_urls = items_urls[:10000]

len(items_urls)

10000

In [10]:
items_urls[0]

'https://kino.mail.ru/series_756333_velikolepnii_vek/'

In [11]:
l1, l2, l3, l4 = items_urls[:2500], items_urls[2500:5000], items_urls[5000:7500], items_urls[7500:10000]

In [12]:
l5 = items_urls[2000:2500]

In [13]:
titles = []
sources = []
descriptions = []
ganres = []
years = []
posters = []
casts = []
ratings = []
s = 1

for url in tqdm(l5):
    try:
        text = requests.get(url).text
        soup = BeautifulSoup(text, 'lxml')
        try:
            title = soup.find('h1', class_='text text_bold_giant color_white').text
        
            titles.append(title[:title.find(' (сери')])
        except:
            print('маслина')
            titles.append(None)
        # тянем постер
        try:
            picture_url = soup.find('meta', itemprop='image')['content']
            posters.append(picture_url)
        except:
            posters.append(None)
        # тянем жанры
        try:
            ganre = soup.find_all('span', class_='badge__text')
            helper = []
            for i in ganre:
                helper.append(i.text)
            ganres.append(helper)
        except:
            ganres.append(None)
        # тянем cast
        try:
            cast = soup.find_all('span', class_='p-truncate__inner js-toggle__truncate-inner')
            a = []
            for act in cast:
                a.append(act.text)
            casts.append(a[1:4])
        except:
            casts.append(None)
        try:
            description = soup.find('div', class_='text text_inline text_light_medium text_fixed valign_baseline p-movie-info__description-text').text
            descriptions.append(description.replace('\xa0', ' '))
        except:
            descriptions.append(None)
        # год
        try:
            year = soup.find('span', class_='nowrap').text
            years.append(year.split()[:3])
        except:
            years.append(None)
        # imdb
        try:
            rating = float(soup.find('span', class_='text text_bold_huge text_fixed').text)
            ratings.append(rating)
        except:
            ratings.append(None)

        sources.append(url)

        if len(sources) % 500 == 0:
            data = pd.DataFrame({'url':sources,
                                'poster':posters,
                                'title':titles,
                                'ganres':ganres,
                                'description':descriptions,
                                'year':years,
                                'rating':ratings,
                                'cast':casts})
            print(f'{len(data)} saved')

            name = f'l1/data_{s}.csv'

            data.to_csv(name)

            s += 1
            titles = []
            sources = []
            descriptions = []
            ganres = []
            years = []
            posters = []
            casts = []
            ratings = []

    except requests.exceptions.RequestException as e:
        print(f"ссылка номер: {items_urls.index(url)}, ссылка: {url}, ошибка: {e}")

    

  2%|▏         | 10/500 [01:14<2:53:19, 21.22s/it]

ссылка номер: 2009, ссылка: https://kino.mail.ru/series_902422_drugoi_maior_sokolov/, ошибка: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 89%|████████▉ | 444/500 [09:28<00:46,  1.21it/s] 

In [13]:
l1[2500]

IndexError: list index out of range

In [8]:
len(l1)

2500